In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.dates as mdates
import pprint
from flask import Flask, jsonify
import numpy as np


# Reflect Tables into SQLAlchemy ORM

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect() 


In [3]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [4]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [5]:
# Save references to each table
Measurements = Base.classes.measurement
Station = Base.classes.station

In [6]:
# Create our session (link) from Python to the DB
session = Session(engine)
first_row = session.query(Measurements).first()
print(first_row.__dict__)

first_row_2 = session.query(Station).first()
print(first_row_2.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1131de748>, 'date': '2010-01-01', 'id': 1, 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1131ec2b0>, 'longitude': -157.8168, 'name': 'WAIKIKI 717.2, HI US', 'id': 1, 'elevation': 3.0, 'latitude': 21.2716, 'station': 'USC00519397'}


In [7]:
latest = session.query(Measurements.date).order_by(Measurements.date.desc()).first()[0]

# Date 12 months before the last date of record
date_start_query = dt.date(2017, 8, 23) - dt.timedelta(days=365)


In [8]:
# Flask Setup
app = Flask(__name__)


@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/&ltstart&gt, for example, /api/v1.0/2017-01-01<br/>"
        f"/api/v1.0/&ltstart&gt/&ltend&gt, for example, /api/v1.0/2017-01-01/2017-01-04"
    )

# Route /api/v1.0/precipitation
@app.route("/api/v1.0/precipitation")
def precipitation():
    
    """
    Return dates and precipitation observations from the last year
    """
    
    results = session.query(Measurement.date, func.avg(Measurement.prcp)).filter(Measurement.date>=date_start_query).group_by(Measurement.date).all()

    results_list = []
    for date, prcp in results:
        results_list.append({str(date): prcp})
    return jsonify(results_list)

# Route /api/v1.0/stations
@app.route("/api/v1.0/stations")
def station():
    
    """
    Return stations
    """
    
    results = session.query(Station.station).all()
    results_list = list(np.ravel(results))

    return jsonify(results_list)

# Route /api/v1.0/tobs
@app.route("/api/v1.0/tobs")
def tobs():
    
    """
    Return dates and temperature observations from the last year
    """
    
    results = session.query(Measurement.date, func.avg(Measurement.tobs)). \
                filter(Measurement.date>=date_start_query). \
                group_by(Measurement.date).all()

    results_list = []
    for date, temp in results:
        results_list.append({str(date): temp})
    return jsonify(results_list)

# Route /api/v1.0/<start>
# Route /api/v1.0/<start>/<end>
@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def start_end(start, end=latest):

    '''
    Return the minimum temperature, the average temperature, and the max temperature for a given start or start-end range
    '''

    start_date_dt = dt.datetime.strptime(start, '%Y-%m-%d')
    if end == latest:
        end_date_dt = end
    else:
        end_date_dt = dt.datetime.strptime(end, '%Y-%m-%d')

    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    temps = session.query(*sel). \
                filter(Measurement.date>=start_date_dt). \
                filter(Measurement.date<=end_date_dt).all()[0]
    
    results_list = [{"temp_min": temps[0]}, 
                    {"temp_avg": temps[1]}, 
                    {"temp_max": temps[2]}]
    return jsonify(results_list)

if __name__ == '__main__':
    app.run(debug=True)

OSError: [Errno 48] Address already in use

In [9]:
# Flask Setup
app = Flask(__name__)


In [10]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/&ltstart&gt, for example, /api/v1.0/2017-01-01<br/>"
        f"/api/v1.0/&ltstart&gt/&ltend&gt, for example, /api/v1.0/2017-01-01/2017-01-04"
    )

In [11]:
# Route /api/v1.0/precipitation
@app.route("/api/v1.0/precipitation")
def precipitation():
    
    """
    Return dates and precipitation observations from the last year
    """
    
    results = session.query(Measurement.date, func.avg(Measurement.prcp)).filter(Measurement.date>=date_start_query).group_by(Measurement.date).all()

    results_list = []
    for date, prcp in results:
        results_list.append({str(date): prcp})
    return jsonify(results_list)

In [12]:
# Route /api/v1.0/stations
@app.route("/api/v1.0/stations")
def station():
    
    """
    Return stations
    """
    
    results = session.query(Station.station).all()
    results_list = list(np.ravel(results))

    return jsonify(results_list)


In [13]:
# Route /api/v1.0/tobs
@app.route("/api/v1.0/tobs")
def tobs():
    
    """
    Return dates and temperature observations from the last year
    """
    
    results = session.query(Measurement.date, func.avg(Measurement.tobs)). \
                filter(Measurement.date>=date_start_query). \
                group_by(Measurement.date).all()

    results_list = []
    for date, temp in results:
        results_list.append({str(date): temp})
    return jsonify(results_list)

In [14]:
# Route /api/v1.0/<start>
# Route /api/v1.0/<start>/<end>
@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def start_end(start, end=latest):

    '''
    Return the minimum temperature, the average temperature, and the max temperature for a given start or start-end range
    '''

    start_date_dt = dt.datetime.strptime(start, '%Y-%m-%d')
    if end == latest:
        end_date_dt = end
    else:
        end_date_dt = dt.datetime.strptime(end, '%Y-%m-%d')

    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    temps = session.query(*sel). \
                filter(Measurement.date>=start_date_dt). \
                filter(Measurement.date<=end_date_dt).all()[0]
    
    results_list = [{"temp_min": temps[0]}, 
                    {"temp_avg": temps[1]}, 
                    {"temp_max": temps[2]}]
    return jsonify(results_list)


In [15]:
if __name__ == '__main__':
    app.run(debug=True)

OSError: [Errno 48] Address already in use